# 第2单元: 使用FrozenLake-v1 ⛄️ 和Taxi-v3 🚕 进行 Q-Learning
在这份笔记中, **你将从头开始编写你的第一个强化学习智能体**使用Q-Learning来玩冰冻湖 ❄️ 并将其分享到社区, 并记得实验不同的配置

❓如果你有任何问题, 请在discord的#study-group-unit2频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境: 
* [FrozenLake-v1](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)
* [Taxi-v3](https://www.gymlibrary.ml/environments/toy_text/taxi/)

📚 强化学习库: Python和NumPy

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![envs.gif](./assets/envs.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用环境库**Gym**.
* 能够从头开始编写一个Q-Learning智能体.
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub**.

## 这份笔记来自深度强化学习课程
![深度强化学习课程.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践**.
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ 在独特的环境中训练**智能体**.

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第2单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit2/README.md)

🔲 📚 [阅读**Q-Learning简介的第1部分**](https://huggingface.co/blog/deep-rl-q-part1)

🔲 📚 [阅读**Q-Learning简介的第2部分**](https://huggingface.co/blog/deep-rl-q-part2)

🔲 📢 注册我们的Discord服务器并在#introduce-yourself频道介绍自己 🥳

🔲 🐕 你是Discord新手吗? 请查看我们的discord 101以获得最佳实践 👉 https://github.com/huggingface/deep-rl-class/blob/main/DISCORD.Md

## 一个Q-Learning的小回顾

* Q-Learning是一种**强化学习算法**
    * 通过训练一个包含**动作(action)-价值(value)的Q-函数(Q-Function)**, Q-表(Q-Table)作为内部存储器, 它包含了**所有的状态-价值对.**
    * 给定一个状态和动作, 我们的Q-函数将**搜索Q-表对应的值.**

![Q-函数-2.jpg](./assets/Q-function-2.jpg)

* 当训练完成时, **我们得到一个最优的Q-函数, 因此得到一个最优的Q-表.**
* 如果我们有**最优的Q-函数**, 那我们就知道**对于每个状态最好的行动是什么**, 所以我们就有一个最优策略.

![link-value-policy.jpg](./assets/link-value-policy.jpg)

但是一开始, 我们的**Q-表是无效的, 因为它的每个状态-动作对都是任意值(大多数时候我们将Q-表初始化为0).** 当我们探索环境并更新我们的Q-表时, 它将给我们越来越好的近似值.

![Q-learning-1.jpg](./assets/Q-learning-1.jpg)

这是Q-Learning的伪代码:

![Q-learning-2.jpg](./assets/Q-learning-2.jpg)

### 第0步: 设置虚拟屏幕 🖥

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境(记录视频帧).**

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [ ]:
!apt install gitlfs ffmpeg 
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### 第1步: 安装依赖项 🔽

第一步是安装多个依赖项:

* `gym`: 包含FrozenLake-v1 ⛄️ 和Taxi-v3 🚕 环境.
* `pygame`: 用于FrozenLake-v1和Taxi-v3的UI.
* `numpy`: 用于处理我们的Q-表.

Hugging Face Hub 🤗 是一个任何人都可以分享和探索模型和数据集的地方. 它有版本控制, 评估, 可视化和其他功能, 可以允许你简单地与他人协作.

你可以在这看到全部可用的深度强化学习模型. 👉 https://huggingface.co/models?other=q-learning

In [ ]:
!pip install gym==0.24.0 pygame
!pip install huggingface_hub
!pip install imageio

### 第2步: 导入包 📦

除了安装的库, 我们还使用:
* `random`: 生成随机数(这对Epsilon-Greedy策略很有用).
* `imageio`: 生成回放视频.

In [ ]:
import os
import random

import gym
import imageio
import numpy as np
import pickle5 as pickle

我们现在已经准备好编写我们的Q-Learing算法了 🔥

# 第1部分: 冰冻湖 ⛄️ (不打滑版)

### 第1步: 创建并理解[冰冻湖环境 ⛄️ ](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)
---
💡 当你开始使用一个环境的好习惯是查看它的文档.

👉 https://www.gymlibrary.ml/environments/toy_text/frozen_lake/

---
我们将训练我们的Q-Learning智能体从起始状态(S)导航到目标状态(G), 方法是仅在冰面(F)上行走并避开冰窟窿(H).

我们有两个尺寸的环境:
* `map_name='4x4'`: 4x4地图版本.
* `map_name='8x8'`: 8x8地图版本.

环境有两种模式:
* `is_slippery=False`: 由于冰冻湖不打滑的特性, 智能体总能沿预期方向移动.
* `is_slippery=True`: 由于冰冻湖打滑(随机)的特性, 智能体可能不总沿预期方向移动.

现在让我们使用4x4地图和不打滑的版本.

In [ ]:
# 创建FrozenLake-v1环境并使用4x4地图和不打滑的版本.
env = gym.make()

### 答案

In [ ]:
env = gym.make(id='FrozenLake-v1',
               map_name='4x4',
               is_slippery=False)

你也可以自定义地图来使用:
```python
desc=['SFFF', 'FHFH', 'FFFH', 'HFFG']
gym.make('FrozenLake-v1', desc=desc, is_slippery=True)
```
但是我们现在将使用默认的环境.

### 让我们一起看看环境是什么样的:

In [ ]:
# 我们使用 gym.make('<环境的名称>') 创建环境.
env.reset()
print('_' * 5 + '可观察的环境' + '_' * 5)
print()
print('可观察的环境', env.observation_space)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

我们通过`可观察的环境 Discrete(16)`看到, 可观察的环境是一个表示**智能体当前位置current_row * nrows + current_col(行和列均从0开始)的值.**

例如, 在4x4的地图上目标位置可以被计算成: 3 * 4 + 3 = 15. 可能的环境值取决于地图的大小. **例如, 4x4的地图有16个可能的环境值.**

对于当前, 初始值 = 0

![image.png](./assets/image.png)

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5)
print()
print('动作的总数', env.action_space.n)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

动作空间(智能体可用的可能动作集)是有4个可用动作的离散值 🎮:

* 0: 向左走
* 1: 向下走
* 2: 向右走
* 3: 向上走

奖励函数 💰:

* 到达目标: +1
* 到达冰窟窿: 0
* 到达冰面: 0

### 第2步: 创建并初始化Q-表 🗄
(👀 伪代码的第1步)
![Q-learning-2.jpg](./assets/Q-learning-2.jpg)
是时候初始化我们的Q-表了! 要知道我们需要使用多少行(states)和列(actions), 我们需要知道可观察和动作空间. OpenAI Gym给我们提供了一个方法: `env.action_space.n`和`env.observation_space.n`

In [ ]:
state_space = 
print('一共有', state_space, '个可能的状态')

action_space = 
print('一共有', action_space, '个可能的动作')

In [ ]:
# 让我们创建大小为(state_space, action_space)的Q-表, 并使用np.zeros将它们的值初始化为0.
def initialize_q_table(state_space, action_space):
    Qtable = 
    return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

### 答案

In [ ]:
state_space = env.observation_space.n
print('一共有', state_space, '个可能的状态')

action_space = env.action_space.n
print('一共有', action_space, '个可能的动作')

In [ ]:
# 让我们创建大小为(state_space, action_space)的Q-表, 并使用np.zeros将它们的值初始化为0.
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros([state_space, action_space])
    return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)